In [1]:
import os

In [3]:
%pwd

'd:\\SANJEEV KUMAR\\END TO END MASTER PROJECT\\text-summerization\\src\\textsummerizer\\components'

In [8]:
os.chdir("../")

In [9]:
%pwd

'd:\\SANJEEV KUMAR\\END TO END MASTER PROJECT\\text-summerization'

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [12]:
from textsummerizer.constants import *
from textsummerizer.utils.common import read_yaml, create_directories

In [13]:
class ConfiggurationManager:
    def __init__(self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_url=config.source_url,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )

        return data_ingestion_config
        

In [14]:
import os
import urllib.request  as request
import zipfile
from textsummerizer.logging import logger
from textsummerizer.utils.common import get_size

In [15]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,header=request.urlretrieve(url=self.config.source_url,
                                                filename= self.config.local_data_file)
            logger.info(f"Downloaded file size: {get_size(self.config.local_data_file)}")
        else:
            logger.info(f"File already exists at {self.config.local_data_file}")

    def extract_zip_file(self):

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Extracted files to {unzip_path}")

In [18]:
try:
    config=ConfiggurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-08-03 23:04:48,102 - textsummerizer - INFO - YAML file config\config.yaml read successfully.]
[2025-08-03 23:04:48,106 - textsummerizer - INFO - YAML file params.yaml read successfully.]
[2025-08-03 23:04:48,107 - textsummerizer - INFO - Directory created: artifacts]
[2025-08-03 23:04:48,109 - textsummerizer - INFO - Directory created: artifacts/data_ingestion]
[2025-08-03 23:04:50,747 - textsummerizer - INFO - Downloaded file size: 7718.35 KB]
[2025-08-03 23:04:50,875 - textsummerizer - INFO - Extracted files to artifacts\data_ingestion]
